### VTT challenge: Innovation Ambiguity

#### Source Dataframe
- Websites from finnish companies that mention 'VTT' on their website
- `Orbis ID`, also `VAT id` is a unique identifier for organizations, later used to merge different alias of the same organization to one unique id

In [1]:
# 1. original source dataframe
import pandas as pd
df = pd.read_csv('data/dataframes/vtt_mentions_comp_domain.csv')
df = df[df['Website'].str.startswith('www.')]
df['source_index'] = df.index

print(f"DF with content from {len(df)} websites of {len(df['Company name'].unique())} different companies ")
df.head(3)

DF with content from 1100 websites of 270 different companies 


,Orbis ID,Company name,Website,Link,Title,date_obtained,Type,text_content,Company_Name_Alias,source_index
0,FI14636114,FORTUM OYJ,www.fortum.com,https://www.fortum.com/media/2020/04/fortum-aw...,Fortum awarded contract for decommissioning of...,2024-09-18,Website,"Press release\n 08 April 2020, 7:00 EEST ...","['Fortum Oyj', 'Fortum Markets AB', 'Fortum Ab...",0
1,FI14636114,FORTUM OYJ,www.fortum.com,https://www.fortum.com/media/2023/03/finnish-g...,Finnish Government grants operating licence fo...,2024-08-16,Website,"Press release\n 30 March 2023, 13:25 EEST ...","['Fortum Oyj', 'Fortum Markets AB', 'Fortum Ab...",1
2,FI14636114,FORTUM OYJ,www.fortum.com,https://www.fortum.com/media/2021/05/apros-sof...,"AprosÂ® software, developed by Fortum and VTT,...",2024-09-12,Website,"Online news\n 21 May 2021, 9:30 EEST ...","['Fortum Oyj', 'Fortum Markets AB', 'Fortum Ab...",2


##### End-to-End relationship extraction
- Based on the above website content, entities of the type `Organization` and `Innovation` are extracted, as well as their type of relationship
- `Collaboration` between Organization and `Developed_by` between Innovation and Organization
- The relationships are stored in a custom object as displayed below: 

In [2]:
# 2.1. example of custom python object of data
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

class Node(BaseModel):
    """Represents a node in the knowledge graph."""
    id: str # unique identifier for node of type 'Organisation', else 'name provided by llm' for type: 'Innovation'
    type: str # allowed node types: 'Organization', 'Innovation'
    properties: Dict[str, str] = Field(default_factory=dict)

class Relationship(BaseModel):
    """Represents a relationship between two nodes in the knowledge graph."""
    source: str
    source_type: str # allowed node types: 'Organization', 'Innovation'
    target: str 
    target_type: str # allowed node types: 'Organization', 'Innovation'
    type: str # allowed relationship types: 'DEVELOPED_BY', 'COLLABORATION'
    properties: Dict[str, str] = Field(default_factory=dict)

class Document(BaseModel):
    page_content:str # manually appended - source text of website
    metadata: Dict[str, str] = Field(default_factory=dict) # metadata including source URL and document ID

class GraphDocument(BaseModel):
    """Represents a complete knowledge graph extracted from a document."""
    nodes: List[Node] = Field(default_factory=list)
    relationships: List[Relationship] = Field(default_factory=list)
    source_document: Optional[Document] = None

# 2.2 loading example of custom graph document
# example file naming convention
print("The extracted graph documents are saved as f'{df['Company name'].replace(' ','_')}_{df['source_index'].pkl}.pkl, under data/graph_docs/ \n")

for i, row in df[:3].iterrows():
    print(f"{i}: 'data/graph_docs/' + {row['Company name'].replace(' ','_')}_{row['source_index']}.pkl")

The extracted graph documents are saved as f'{df['Company name'].replace(' ','_')}_{df['source_index'].pkl}.pkl, under data/graph_docs/ 

0: 'data/graph_docs/' + FORTUM_OYJ_0.pkl
1: 'data/graph_docs/' + FORTUM_OYJ_1.pkl
2: 'data/graph_docs/' + FORTUM_OYJ_2.pkl


### CSV to PKL Extraction Process (Demo)
Below is an example showing how the graph documents were originally created from CSV text using LLM extraction.

**Note:** This is a demonstration of the extraction pipeline. The actual PKL files in `data/graph_docs/` were pre-generated using a similar process.

In [3]:
# CSV to PKL Extraction Demo - Complete Pipeline
import pickle
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document as LangChainDocument

def extract_graph_from_text(text: str, source_url: str, company_name: str, llm_model) -> GraphDocument:
    """
    使用 LLM 从文本中提取知识图谱
    
    Args:
        text: 网站文本内容
        source_url: 来源 URL
        company_name: 公司名称
        llm_model: 已初始化的 LLM 模型
        
    Returns:
        GraphDocument: 提取的图结构
    """
    # 创建 LangChain Document
    doc = LangChainDocument(
        page_content=text,
        metadata={
            'source_url': source_url,
            'company': company_name
        }
    )
    
    # 配置图转换器 - 指定允许的节点类型和关系类型
    graph_transformer = LLMGraphTransformer(
        llm=llm_model,
        allowed_nodes=["Organization", "Innovation"],  # 只提取这两种类型的实体
        allowed_relationships=["DEVELOPED_BY", "COLLABORATION"],  # 只提取这两种关系
        node_properties=["description"],  # 为节点提取描述属性
        relationship_properties=["description"]  # 为关系提取描述属性
    )
    
    print("🤖 正在调用 LLM 提取实体和关系...")
    
    # 使用 LLM 提取图结构
    langchain_graph_docs = graph_transformer.convert_to_graph_documents([doc])
    
    if not langchain_graph_docs:
        print("⚠️ 未提取到任何图结构")
        return None
    
    # 转换为我们的自定义 GraphDocument 格式
    langchain_graph = langchain_graph_docs[0]
    
    # 转换节点
    custom_nodes = []
    for node in langchain_graph.nodes:
        custom_node = Node(
            id=node.id,
            type=node.type,
            properties={
                'description': node.id,  # 使用 id 作为描述
                'english_id': node.id    # 英文 ID
            }
        )
        custom_nodes.append(custom_node)
    
    # 转换关系
    custom_relationships = []
    for rel in langchain_graph.relationships:
        custom_rel = Relationship(
            source=rel.source.id,
            source_type=rel.source.type,
            target=rel.target.id,
            target_type=rel.target.type,
            type=rel.type,
            properties={'description': f"{rel.source.id} {rel.type} {rel.target.id}"}
        )
        custom_relationships.append(custom_rel)
    
    # 创建 Document
    source_doc = Document(
        page_content=text,
        metadata={'source_url': source_url, 'company': company_name}
    )
    
    # 创建最终的 GraphDocument
    graph_doc = GraphDocument(
        nodes=custom_nodes,
        relationships=custom_relationships,
        source_document=source_doc
    )
    
    print(f"✅ 提取完成: {len(custom_nodes)} 个节点, {len(custom_relationships)} 个关系")
    
    return graph_doc


def save_graph_to_pkl(graph_doc: GraphDocument, output_path: str):
    """保存 GraphDocument 到 PKL 文件"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'wb') as f:
        pickle.dump([graph_doc], f)  # 注意：保存为列表格式以匹配现有文件结构
    print(f"💾 已保存到: {output_path}")


# 测试函数：处理 DataFrame 中的一行数据
def process_single_row(row, llm_model, output_dir='data/graph_docs_demo/'):
    """
    处理单行数据的完整流程
    
    Args:
        row: DataFrame 的一行
        llm_model: LLM 模型
        output_dir: 输出目录
    """
    print(f"\n{'='*60}")
    print(f"📄 处理公司: {row['Company name']}")
    print(f"🔗 来源: {row['Link']}")
    print(f"{'='*60}\n")
    
    # 提取图结构
    graph_doc = extract_graph_from_text(
        text=row['text_content'],
        source_url=row['Link'],
        company_name=row['Company name'],
        llm_model=llm_model
    )
    
    if graph_doc is None:
        return None
    
    # 生成文件名
    filename = f"{row['Company name'].replace(' ', '_')}_{row['source_index']}.pkl"
    output_path = os.path.join(output_dir, filename)
    
    # 保存
    save_graph_to_pkl(graph_doc, output_path)
    
    return graph_doc

print("✨ 提取管道函数已定义完成！")
print("\n使用方法:")
print("graph_doc = process_single_row(df.iloc[0], model)")


✨ 提取管道函数已定义完成！

使用方法:
graph_doc = process_single_row(df.iloc[0], model)


In [8]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv

def initialize_llm(deployment_model: str = None) -> AzureChatOpenAI:
    """
    从 .env 文件初始化 Azure OpenAI 模型
    
    Args:
        deployment_model: 可选，指定部署模型名称（如 'gpt-4o-mini'）
                        如果不指定，会使用 .env 中的 AZURE_OPENAI_DEPLOYMENT
    
    Returns:
        AzureChatOpenAI: 初始化好的 LLM 模型
    """
    # 加载 .env 文件
    load_dotenv()
    
    # 从环境变量读取配置
    api_key = os.getenv('AZURE_OPENAI_API_KEY')
    endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
    api_version = os.getenv('AZURE_OPENAI_API_VERSION')
    
    # 如果没有指定模型，使用环境变量中的默认模型
    if deployment_model is None:
        deployment_model = os.getenv('AZURE_OPENAI_DEPLOYMENT', 'gpt-4o-mini')
    
    # 验证必需的配置
    if not api_key:
        raise ValueError("❌ AZURE_OPENAI_API_KEY 未在 .env 文件中配置")
    if not endpoint:
        raise ValueError("❌ AZURE_OPENAI_ENDPOINT 未在 .env 文件中配置")
    if not api_version:
        raise ValueError("❌ AZURE_OPENAI_API_VERSION 未在 .env 文件中配置")
    
    print(f"✅ 从 .env 加载配置:")
    print(f"   模型: {deployment_model}")
    print(f"   端点: {endpoint}")
    print(f"   API 版本: {api_version}")
    
    return AzureChatOpenAI(
        model=deployment_model,
        api_key=api_key,
        azure_endpoint=endpoint,
        api_version=api_version
    )

#### 🔑 配置说明

在运行下面的代码之前，请确保：

1. **创建 `.env` 文件**（如果还没有）：
   ```bash
   # 从模板复制
   cp .env.template .env
   ```

2. **编辑 `.env` 文件，填入你的 Azure OpenAI 配置**：
   ```
   AZURE_OPENAI_API_KEY=你的API密钥
   AZURE_OPENAI_ENDPOINT=https://你的资源名.openai.azure.com/
   AZURE_OPENAI_API_VERSION=2024-08-01-preview
   AZURE_OPENAI_DEPLOYMENT=gpt-4o-mini
   ```

3. **保存 `.env` 文件**，然后运行下面的 cell

💡 **新版本的优势**：
- ✅ 不再需要 JSON 配置文件
- ✅ 直接从 `.env` 读取配置
- ✅ 配置更简单，一个地方管理所有环境变量

In [9]:
# 🚀 运行测试：处理第一行数据
# 注意：确保 .env 文件已配置好 Azure OpenAI 的 API 密钥和端点

# 初始化模型（从 .env 读取配置）
try:
    model = initialize_llm(deployment_model='gpt-4o-mini')  # 或不传参数使用 .env 中的默认模型
    print("✅ LLM 模型初始化成功")
except Exception as e:
    print(f"❌ 模型初始化失败: {e}")
    print("请确保 .env 文件存在，并且包含有效的配置信息")

# 处理第一行数据
try:
    test_row = df.iloc[0]
    print(f"\n测试数据:")
    print(f"  公司: {test_row['Company name']}")
    print(f"  URL: {test_row['Link']}")
    print(f"  文本长度: {len(test_row['text_content'])} 字符")
    
    # 执行提取
    result_graph = process_single_row(test_row, model, output_dir='data/graph_docs_demo/')
    
    if result_graph:
        print("\n📊 提取结果预览:")
        print(f"\n节点 ({len(result_graph.nodes)}):")
        for node in result_graph.nodes[:5]:  # 显示前5个
            print(f"  - {node.id} ({node.type})")
        
        print(f"\n关系 ({len(result_graph.relationships)}):")
        for rel in result_graph.relationships[:5]:  # 显示前5个
            print(f"  - {rel.source} --[{rel.type}]--> {rel.target}")
    
except Exception as e:
    print(f"\n❌ 处理失败: {e}")
    import traceback
    traceback.print_exc()

✅ 从 .env 加载配置:
   模型: gpt-4o-mini
   端点: https://lijie-mazglg3v-eastus2.cognitiveservices.azure.com/
   API 版本: 2025-01-01-preview
✅ LLM 模型初始化成功

测试数据:
  公司: FORTUM OYJ
  URL: https://www.fortum.com/media/2020/04/fortum-awarded-contract-decommissioning-vtts-nuclear-research-reactor-finland
  文本长度: 2246 字符

📄 处理公司: FORTUM OYJ
🔗 来源: https://www.fortum.com/media/2020/04/fortum-awarded-contract-decommissioning-vtts-nuclear-research-reactor-finland

🤖 正在调用 LLM 提取实体和关系...
✅ 提取完成: 4 个节点, 3 个关系
💾 已保存到: data/graph_docs_demo/FORTUM_OYJ_0.pkl

📊 提取结果预览:

节点 (4):
  - Fortum Corporation (Organization)
  - Vtt Technical Research Centre Of Finland Ltd (Organization)
  - Fir1 Research Nuclear Reactor (Innovation)
  - Radiation And Nuclear Safety Authority Of Finland (Stuk) (Organization)

关系 (3):
  - Fortum Corporation --[COLLABORATION]--> Vtt Technical Research Centre Of Finland Ltd
  - Fortum Corporation --[DEVELOPED_BY]--> Fir1 Research Nuclear Reactor
  - Fir1 Research Nuclear Reactor --[COLLABORA

In [13]:
# 🔍 验证环境：确认 Kernel 使用的是 Container 环境
import sys
import os
import platform

print("=" * 60)
print("📍 当前执行环境信息")
print("=" * 60)

print(f"\n1️⃣ Python 解释器:")
print(f"   路径: {sys.executable}")
print(f"   版本: {sys.version}")

print(f"\n2️⃣ 操作系统:")
print(f"   系统: {platform.system()}")
print(f"   版本: {platform.version()}")
print(f"   架构: {platform.machine()}")

print(f"\n3️⃣ 主机名 (Container ID):")
print(f"   {platform.node()}")

print(f"\n4️⃣ 当前工作目录:")
print(f"   {os.getcwd()}")

print(f"\n5️⃣ 关键包版本:")


print(f"\n6️⃣ 环境类型:")
if '/usr/local/bin/python' in sys.executable:
    print(f"   ✅ 正在使用 Container 中的系统 Python")
elif 'conda' in sys.executable:
    print(f"   📦 正在使用 Conda 环境")
else:
    print(f"   ❓ 其他环境: {sys.executable}")

print("\n" + "=" * 60)
print("结论: Jupyter Kernel 运行在 Dev Container 内部！")
print("=" * 60)

📍 当前执行环境信息

1️⃣ Python 解释器:
   路径: /usr/local/bin/python
   版本: 3.11.13 (main, Jul  2 2025, 01:25:53) [GCC 10.2.1 20210110]

2️⃣ 操作系统:
   系统: Linux
   版本: #1 SMP Sat May 17 08:28:57 UTC 2025
   架构: aarch64

3️⃣ 主机名 (Container ID):
   507c52292a80

4️⃣ 当前工作目录:
   /workspaces/Innovation-Duplication

5️⃣ 关键包版本:

6️⃣ 环境类型:
   ✅ 正在使用 Container 中的系统 Python

结论: Jupyter Kernel 运行在 Dev Container 内部！


#### 💡 Dev Container vs 本地环境对比

下面的代码会显示 Container 和你本地环境的区别：

In [14]:
# 🔬 对比：Container 环境 vs 你的本地环境
import subprocess
import sys
import os

print("=" * 70)
print("🔍 当前运行环境（Container 内部）")
print("=" * 70)

print("\n📍 操作系统:")
try:
    result = subprocess.run(['cat', '/etc/os-release'], capture_output=True, text=True)
    for line in result.stdout.split('\n')[:3]:
        if line:
            print(f"   {line}")
except:
    print("   无法读取")

print("\n📍 Python 位置:")
print(f"   {sys.executable}")
print(f"   版本: {sys.version.split()[0]}")

print("\n📍 当前用户:")
result = subprocess.run(['whoami'], capture_output=True, text=True)
print(f"   {result.stdout.strip()}")

print("\n📍 主机名 (Container ID):")
result = subprocess.run(['hostname'], capture_output=True, text=True)
print(f"   {result.stdout.strip()}")

print("\n📍 文件系统根目录:")
result = subprocess.run(['ls', '-la', '/'], capture_output=True, text=True)
print("   这是 Container 的独立文件系统，不是你电脑的 C:/ 或 /")

print("\n📍 可用的命令:")
commands_to_check = ['python3', 'pip3', 'git', 'docker']
for cmd in commands_to_check:
    result = subprocess.run(['which', cmd], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✅ {cmd}: {result.stdout.strip()}")
    else:
        print(f"   ❌ {cmd}: 未安装")

print("\n" + "=" * 70)
print("💡 关键理解:")
print("=" * 70)
print("""
1. 这是一个 **独立的 Linux 虚拟环境**（Debian 11）
2. Python 3.11.13 是在 Container 内部安装的，不是你本地的
3. 所有包都是重新安装的，不使用你电脑上的任何环境
4. 但是项目文件是**共享的**（你编辑，Container 能看到）

类比：
- Container = 云服务器上的虚拟机
- 你的代码运行在"云端"（Container）
- 但你可以在本地（VS Code）编辑和查看结果
""")

🔍 当前运行环境（Container 内部）

📍 操作系统:
   PRETTY_NAME="Debian GNU/Linux 11 (bullseye)"
   NAME="Debian GNU/Linux"
   VERSION_ID="11"

📍 Python 位置:
   /usr/local/bin/python
   版本: 3.11.13

📍 当前用户:
   vscode

📍 主机名 (Container ID):
   507c52292a80

📍 文件系统根目录:
   这是 Container 的独立文件系统，不是你电脑的 C:/ 或 /

📍 可用的命令:
   ✅ python3: /usr/local/bin/python3
   ✅ pip3: /usr/local/bin/pip3
   ✅ git: /usr/local/bin/git
   ❌ docker: 未安装

💡 关键理解:

1. 这是一个 **独立的 Linux 虚拟环境**（Debian 11）
2. Python 3.11.13 是在 Container 内部安装的，不是你本地的
3. 所有包都是重新安装的，不使用你电脑上的任何环境
4. 但是项目文件是**共享的**（你编辑，Container 能看到）

类比：
- Container = 云服务器上的虚拟机
- 你的代码运行在"云端"（Container）
- 但你可以在本地（VS Code）编辑和查看结果



In [16]:
# 2.3 loading example of custom graph document

import pickle, os
path = 'data/graph_docs/'
index = 0

# load graph document

with open(os.path.join(path, os.listdir(path)[index]), 'rb') as doc:
    graph_doc = pickle.load(doc)

print(f"Example custom graph document:\n\n {graph_doc} \n\n ")
print("Example custom graph document nodes :\n")
for doc in graph_doc:
    for node in doc.nodes:
        print(f"- {node.id} ({node.type})    :   {node.properties['description']}")

print("\nExample custom graph document relationships:\n")
for doc in graph_doc:
    for relationship in doc.relationships:
        print(f"- {relationship.source} ({relationship.source_type}) - {relationship.type} -> {relationship.target} ({relationship.target_type})    :    description: {relationship.properties['description']}")


Example custom graph document:

 [GraphDocument(nodes=[], relationships=[], source_document=Document(metadata={'source_id': 'TS-YHTYMA OY_797'}, page_content='\r\nTurun Sanomat                              Teknologian tutkimuskeskus VTT on nimittÃ¤nyt hallitukseensa Turun yliopiston rehtorin Jukka Kolan. Kola valittiin tehtÃ¤vÃ¤Ã¤n VTT:n yhtiÃ¶kokouksessa. Toisena uutena jÃ¤senenÃ¤ hallitukseen valittiin Futuricen toimitusjohtaja Teemu Moisala. VTT:n hallituksen puheenjohtajana toimii Suomen ABB:n toimitusjohtaja Pekka Tiitinen. Hallituksessa jatkavat Pekka Tiitisen lisÃ¤ksi teknologiajohtaja Heli Antila Fortum, talousjohtaja Harri LeiviskÃ¤ Dayton Groupista, toimitusjohtaja Matti Hietanen (Suomen Malmijalostuksesta ja osastopÃ¤Ã¤llikkÃ¶ Marja-Riitta Pihlman tyÃ¶- ja elinkeinoministeriÃ¶stÃ¤. Seuraa ja lue uutisia aiheesta Huomio! Tutustu ennen kommentoimista Turun Sanomien keskustelupalstan sÃ¤Ã¤ntÃ¶ihin tÃ¤Ã¤llÃ¤. Uudet nÃ¤kÃ¶kulmat keskustelussa vievÃ¤t asioita eteenpÃ¤in. Siksi Tur

#### Name ambiguity resolution (名称消歧)

**问题：** 在原始文本中，同一个组织可能有多种名称变体/别名，导致歧义

**解决方案：** 通过将组织映射到唯一标识符 `VAT ID`（芬兰企业税号）来部分解决这个问题

---

### 📁 两个目录的区别：

#### 1️⃣ `data/graph_docs/` - **原始版本**
- LLM 直接从文本中提取的实体名称
- **Organization 节点的 ID** = LLM 提取的公司名称（如 "AINS Group", "VTT Technical Research Centre of Finland"）
- ❌ 问题：同一家公司可能有多个不同的名称表述

**示例节点：**
```
节点 ID: "AINS Group"  
节点 ID: "VTT Technical Research Centre of Finland"
```

#### 2️⃣ `data/graph_docs_names_resolved/` - **名称解析后版本**  
- **Organization 节点的 ID** = 唯一的 VAT ID（如 "FI31646146", "FI26473754"）
- ✅ 优势：相同公司的所有别名都映射到同一个 VAT ID
- 节点的 `properties['english_id']` 保留了原始英文名称

**示例节点：**
```
节点 ID: "FI31646146"  (原名: AINS Group)
节点 ID: "FI26473754"  (原名: VTT Technical Research Centre of Finland)
```

---

### 🔑 `entity_glossary` 的作用：

这是一个字典，存储了 **VAT ID → [所有别名列表]** 的映射关系

```python
{
  "FI26473754": ["VTT", "VTT Technical Research Centre", "VTT Finland", ...]
}
```

这样就可以把所有提到 "VTT"、"VTT Finland" 等的地方统一识别为同一个组织！

In [17]:
# 🔍 对比示例：查看名称解析前后的区别
import pickle
import os

# 加载同一个公司的两个版本
filename = 'A-INSINOORIT_OY_638.pkl'

# 原始版本
with open(os.path.join('data/graph_docs/', filename), 'rb') as f:
    graph_original = pickle.load(f)[0]

# 名称解析后版本
with open(os.path.join('data/graph_docs_names_resolved/', filename), 'rb') as f:
    graph_resolved = pickle.load(f)[0]

print("=" * 70)
print("📊 原始版本 (graph_docs) - Organization 节点:")
print("=" * 70)
for node in graph_original.nodes:
    if node.type == 'Organization':
        print(f"\n✏️  节点 ID: {node.id}")
        print(f"   描述: {node.properties.get('description', 'N/A')[:100]}...")

print("\n" + "=" * 70)
print("📊 名称解析后 (graph_docs_names_resolved) - Organization 节点:")
print("=" * 70)
for node in graph_resolved.nodes:
    if node.type == 'Organization':
        print(f"\n🔑 节点 ID: {node.id}  (VAT ID)")
        print(f"   原始英文名: {node.properties.get('english_id', 'N/A')}")
        print(f"   描述: {node.properties.get('description', 'N/A')[:100]}...")

print("\n" + "=" * 70)
print("💡 关键区别总结:")
print("=" * 70)
print("""
原始版本: 节点 ID = "AINS Group", "VTT Technical Research Centre"
         ❌ 如果文本中写 "VTT" 和 "VTT Finland"，会被识别为不同节点

解析后版本: 节点 ID = "FI31646146", "FI26473754" (VAT ID)
           ✅ 所有 VTT 的变体都映射到同一个 VAT ID: FI26473754
           ✅ 消除了名称歧义，便于后续分析
""")

📊 原始版本 (graph_docs) - Organization 节点:

✏️  节点 ID: AINS Group
   描述: A company conducting urban development research and analysis, including the mentioned study on green...

✏️  节点 ID: VTT Technical Research Centre of Finland
   描述: A research partner organization collaborating with AINS Group on the analysis of green areas impact ...

✏️  节点 ID: City of Helsinki
   描述: City participating as a partner in the project concerning urban green areas and housing prices....

✏️  节点 ID: City of Espoo
   描述: City participating as a partner in the project concerning urban green areas and housing prices....

📊 名称解析后 (graph_docs_names_resolved) - Organization 节点:

🔑 节点 ID: FI31646146  (VAT ID)
   原始英文名: AINS Group
   描述: A company conducting urban development research and analysis, including the mentioned study on green...

🔑 节点 ID: FI26473754  (VAT ID)
   原始英文名: VTT Technical Research Centre of Finland
   描述: A research partner organization collaborating with AINS Group on the analysis of green a

In [15]:
# 3. load entity glossary
import json
entity_glossary = json.load(open('data/entity_glossary/entity_glossary.json', 'r', encoding = 'utf-8'))
print(entity_glossary.get('FI26473754'))

{'alias': ['Valtion teknillinen tutkimuskeskus', 'VTT:ltä', 'Teknologian tutkimuskeskus VTT:llÃ¤', 'VTT â\x80\x93 beyond the obvious', 'VTT Oy', 'VTT TECHNICAL RESEAR CH CENTRE OF FINLAND LT D', 'VTT Technical Research Centres of Finland', 'VTT iBEX', 'Tehnologian tutkimuskeskus VTT', 'VTT:n', 'VTT technical research centre', 'VTT Technical Research Centre of Finland Ltd.', 'VTT:ltÃ¤', 'VTT (Technical Research Center of Finland)', 'Teknologian tutkimuskeskus VTT Oy', 'centre de recherche technique de Finlande', 'VTT:llÃ¤', 'Teknologian tutkimuskeskus VTT', 'TEKNOLOGIAN TUTKIMUSKESKUS VTT OY', 'VTT T echnical Research Centre of Finland', 'VTT TECHNICAL RESEARCH CENTRE OF FINLAND LTD', 'Technical Research Centre VTT', 'VTT Technical Research Centre', 'VTT Technical Research Centre in Finland', 'VTT Technical Research Centre of Finland Ltd', 'VTT-R-0025 5-20', 'VTT:t\x0224', 'VTT:llä', 'VTT LaunchPad', 'Technologian tutkimuskeskus VTT Oy', 'Facebook, LinkedIn, YouTube and Instagram', 'Tek

In [18]:
# 2.3 loading example of custom graph document

import pickle, os
path = 'data/graph_docs_names_resolved/'
index = 0

# load graph document
with open(os.path.join(path, os.listdir(path)[index]), 'rb') as doc:
    graph_doc = pickle.load(doc)

print(f"Example custom graph document:\n\n {graph_doc} \n\n ")

print("Example custom graph document nodes :\n")
for doc in graph_doc:
    for node in doc.nodes[:3]:
        print(f"- {node.id} ({node.type})    :   {node.properties['description']}")

print("\nExample custom graph document relationships:\n")
for doc in graph_doc:
    for relationship in doc.relationships[:3]:
        print(f"- {relationship.source} ({relationship.source_type}) - {relationship.type} -> {relationship.target} ({relationship.target_type})    :    description: {relationship.properties['description']}")


Example custom graph document:

 [GraphDocument(nodes=[], relationships=[], source_document=Document(metadata={'source_id': 'TS-YHTYMA OY_797'}, page_content='\r\nTurun Sanomat                              Teknologian tutkimuskeskus VTT on nimittÃ¤nyt hallitukseensa Turun yliopiston rehtorin Jukka Kolan. Kola valittiin tehtÃ¤vÃ¤Ã¤n VTT:n yhtiÃ¶kokouksessa. Toisena uutena jÃ¤senenÃ¤ hallitukseen valittiin Futuricen toimitusjohtaja Teemu Moisala. VTT:n hallituksen puheenjohtajana toimii Suomen ABB:n toimitusjohtaja Pekka Tiitinen. Hallituksessa jatkavat Pekka Tiitisen lisÃ¤ksi teknologiajohtaja Heli Antila Fortum, talousjohtaja Harri LeiviskÃ¤ Dayton Groupista, toimitusjohtaja Matti Hietanen (Suomen Malmijalostuksesta ja osastopÃ¤Ã¤llikkÃ¶ Marja-Riitta Pihlman tyÃ¶- ja elinkeinoministeriÃ¶stÃ¤. Seuraa ja lue uutisia aiheesta Huomio! Tutustu ennen kommentoimista Turun Sanomien keskustelupalstan sÃ¤Ã¤ntÃ¶ihin tÃ¤Ã¤llÃ¤. Uudet nÃ¤kÃ¶kulmat keskustelussa vievÃ¤t asioita eteenpÃ¤in. Siksi Tur

In [19]:
# 📊 将图文档 (Graph Documents) 转换为 DataFrame 表格
import pandas as pd
from tqdm import tqdm

# 创建空的 DataFrame 用于存储所有关系
df_relationships_comp_url = pd.DataFrame(index=None)

# 使用进度条遍历所有公司数据
with tqdm(total=len(df), desc="Entities resolved") as pbar:
    for i, row in df.iterrows(): 
        try:     
            # 1️⃣ 加载单个公司的图文档（PKL 文件）
            Graph_Docs = pickle.load(
                open(os.path.join('data/graph_docs_names_resolved/', 
                                  f"{row['Company name'].replace(' ','_')}_{i}.pkl"), 'rb')
            )[0]
                
            # 2️⃣ 构建节点 ID 到属性的映射字典（方便查询）
            node_description = {}  # 节点 ID -> 描述
            node_en_id = {}        # 节点 ID -> 英文名称
            for node in Graph_Docs.nodes:
                node_description[node.id] = node.properties['description']
                node_en_id[node.id] = node.properties['english_id']

            # 3️⃣ 遍历所有关系，将每个关系转换为一行数据
            relationship_rows = []
            for i in range(len(Graph_Docs.relationships)):
                # 获取当前关系
                rel = Graph_Docs.relationships[i]
                
                # 创建一行数据，包含关系的所有信息
                relationship_rows.append({
                    # 元信息
                    "Document number": row['source_index'],
                    "Source Company": row["Company name"],
                    "relationship description": rel.properties['description'],
                    
                    # 起点节点信息（source）
                    "source id": rel.source,                                    # VAT ID 或 Innovation 名称
                    "source type": rel.source_type,                             # Organization/Innovation
                    "source english_id": node_en_id.get(rel.source, None),      # 原始英文名
                    "source description": node_description.get(rel.source, None), # 节点描述
                    
                    # 关系类型
                    "relationship type": rel.type,                              # COLLABORATION/DEVELOPED_BY
                    
                    # 终点节点信息（target）
                    "target id": rel.target,
                    "target type": rel.target_type,
                    "target english_id": node_en_id.get(rel.target, None),
                    "target description": node_description.get(rel.target, None),
                    
                    # 原始数据来源
                    "Link Source Text": row["Link"],
                    "Source Text": row["text_content"],
                })

            # 4️⃣ 将当前公司的所有关系合并到总 DataFrame
            df_relationships_comp_url = pd.concat([
                df_relationships_comp_url, 
                pd.DataFrame(relationship_rows, index=None)
            ], ignore_index=True)

        except:
            # 如果文件不存在或读取失败，跳过
            continue
        
        # 更新进度条
        pbar.update(1)

# 5️⃣ 显示结果的前几行
df_relationships_comp_url.head(5)

Entities resolved:  96%|█████████▌| 1053/1100 [00:01<00:00, 753.80it/s]


,Document number,Source Company,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text
0,0,FORTUM OYJ,Fortum Corporation is responsible for executin...,First nuclear decommissioning project in Finla...,Innovation,First nuclear decommissioning project in Finla...,The first nuclear reactor decommissioning proj...,DEVELOPED_BY,FI14636114,Organization,Fortum Corporation,A company with over 40 years of experience in ...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."
1,0,FORTUM OYJ,VTT Technical Research Centre of Finland Ltd c...,First nuclear decommissioning project in Finla...,Innovation,First nuclear decommissioning project in Finla...,The first nuclear reactor decommissioning proj...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd,A research organization in Finland contracting...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."
2,0,FORTUM OYJ,Fortum Corporation and VTT Technical Research ...,FI14636114,Organization,Fortum Corporation,A company with over 40 years of experience in ...,COLLABORATION,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd,A research organization in Finland contracting...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."
3,0,FORTUM OYJ,Fortum Corporation has been awarded and is con...,FiR1 nuclear reactor decommissioning,Innovation,FiR1 nuclear reactor decommissioning,"The process of planning, preparatory measures,...",DEVELOPED_BY,FI14636114,Organization,Fortum Corporation,A company with over 40 years of experience in ...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."
4,0,FORTUM OYJ,The Radiation and Nuclear Safety Authority of ...,FiR1 nuclear reactor decommissioning,Innovation,FiR1 nuclear reactor decommissioning,"The process of planning, preparatory measures,...",DEVELOPED_BY,temp_1,Organization,Radiation and Nuclear Safety Authority of Finl...,The national regulatory authority supervising ...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."


#### 📊 将图文档转换为 DataFrame（表格格式）

下面的代码将所有的 PKL 图文档转换为一个统一的关系表格（DataFrame），方便后续分析。

**转换流程：**
1. **遍历所有公司的图文档**：从 `graph_docs_names_resolved/` 读取每个 PKL 文件
2. **提取关系三元组**：每个关系包含 `(source) --[关系类型]--> (target)`
3. **扁平化存储**：将图结构转换为表格行，每行代表一个关系
4. **合并所有数据**：将所有公司的关系合并到一个大 DataFrame 中

**最终 DataFrame 的列：**
- `Document number`: 源文档编号
- `Source Company`: 来源公司名称
- `source id`: 关系起点节点的 ID（VAT ID 或 Innovation 名称）
- `source type`: 起点节点类型（Organization/Innovation）
- `source english_id`: 起点节点的英文原始名称
- `source description`: 起点节点的描述
- `relationship type`: 关系类型（COLLABORATION/DEVELOPED_BY）
- `target id`: 关系终点节点的 ID
- `target type`: 终点节点类型
- `target english_id`: 终点节点的英文原始名称
- `target description`: 终点节点的描述
- `Link Source Text`: 原始网页 URL
- `Source Text`: 原始网页文本内容

**为什么需要这个转换？**
- ✅ 方便用 pandas 进行数据分析和筛选
- ✅ 可以轻松统计每个组织的合作次数
- ✅ 便于导出到 CSV 或其他格式
- ✅ 支持 SQL 风格的查询和聚合操作

In [20]:
# 🔍 示例：理解转换过程
import pickle
import os

# 加载一个示例图文档
filename = 'A-INSINOORIT_OY_638.pkl'
with open(os.path.join('data/graph_docs_names_resolved/', filename), 'rb') as f:
    example_graph = pickle.load(f)[0]

print("=" * 70)
print("📦 原始图结构 (GraphDocument)")
print("=" * 70)
print(f"节点数: {len(example_graph.nodes)}")
print(f"关系数: {len(example_graph.relationships)}")

print("\n示例关系 (图结构):")
for i, rel in enumerate(example_graph.relationships[:2]):
    print(f"\n关系 {i+1}:")
    print(f"  {rel.source} ({rel.source_type})")
    print(f"    --[{rel.type}]-->")
    print(f"  {rel.target} ({rel.target_type})")

print("\n" + "=" * 70)
print("📊 转换后的表格结构 (DataFrame Row)")
print("=" * 70)
print("每个关系变成表格中的一行，包含以下列：\n")

# 模拟一行数据
if example_graph.relationships:
    rel = example_graph.relationships[0]
    sample_row = {
        "source id": rel.source,
        "source type": rel.source_type,
        "relationship type": rel.type,
        "target id": rel.target,
        "target type": rel.target_type,
    }
    
    for key, value in sample_row.items():
        print(f"  {key:20s}: {value}")

print("\n💡 优势：")
print("  ✅ 可以用 pandas 筛选、排序、聚合")
print("  ✅ 方便统计分析（如：哪个组织合作最多？）")
print("  ✅ 易于导出为 CSV、Excel 等格式")
print("  ✅ 支持 SQL 风格的查询操作")

📦 原始图结构 (GraphDocument)
节点数: 8
关系数: 7

示例关系 (图结构):

关系 1:
  Analysis of urban green areas impact on attractiveness and housing prices (Innovation)
    --[DEVELOPED_BY]-->
  FI31646146 (Organization)

关系 2:
  Analysis of urban green areas impact on attractiveness and housing prices (Innovation)
    --[DEVELOPED_BY]-->
  FI26473754 (Organization)

📊 转换后的表格结构 (DataFrame Row)
每个关系变成表格中的一行，包含以下列：

  source id           : Analysis of urban green areas impact on attractiveness and housing prices
  source type         : Innovation
  relationship type   : DEVELOPED_BY
  target id           : FI31646146
  target type         : Organization

💡 优势：
  ✅ 可以用 pandas 筛选、排序、聚合
  ✅ 方便统计分析（如：哪个组织合作最多？）
  ✅ 易于导出为 CSV、Excel 等格式
  ✅ 支持 SQL 风格的查询操作


#### 🔄 数据转换流程图解

```
📦 图文档 (PKL 文件)                    📊 DataFrame (表格)
━━━━━━━━━━━━━━━━━━━                    ━━━━━━━━━━━━━━━━━━━━━━━━━

GraphDocument {                        | source_id | type  | target_id |
  nodes: [                             |-----------|-------|-----------|
    Node(                              | FI123     | COLLAB| FI456     |
      id="FI123",                      | FI456     | DEVEL | Innov1    |
      type="Organization"              | FI789     | COLLAB| FI123     |
    ),                                 | ...       | ...   | ...       |
    Node(id="FI456", ...),             
    Innovation(id="Innov1", ...)       
  ],
  relationships: [
    Relationship(
      source="FI123",                  每个关系 → 一行数据
      type="COLLABORATION",            扁平化存储，便于查询
      target="FI456"
    ),
    ...
  ]
}

🔍 好处：
   图结构 → 难以查询统计
   表格    → 易于分析、聚合、可视化
```

**转换示例：**
```python
# 图结构查询（复杂）
for rel in graph.relationships:
    if rel.source == "FI26473754" and rel.type == "COLLABORATION":
        print(rel.target)

# DataFrame 查询（简单！）
df[(df['source id'] == 'FI26473754') & 
   (df['relationship type'] == 'COLLABORATION')]['target id']
```

#### 🌐 VTT 域名下的创新与合作披露（第二个数据源）

除了在**公司网站**上讨论 VTT 的贡献外，第二个数据源包括 **VTT 域名下的网站**，这些网站讨论了与其他公司的合作。

---

### 📊 两个数据源的对比：

| 特性 | 数据源 1️⃣ (前面处理的) | 数据源 2️⃣ (当前处理的) |
|------|---------------------|---------------------|
| **网站来源** | 芬兰公司的官网 | VTT 官方域名下的页面 |
| **CSV 文件** | `vtt_mentions_comp_domain.csv` | `comp_mentions_vtt_domain.csv` |
| **原始图文档** | `graph_docs/` | `graph_docs_vtt_domain/` |
| **解析后图文档** | `graph_docs_names_resolved/` | `graph_docs_vtt_domain_names_resolved/` |
| **视角** | 公司视角：公司如何描述与 VTT 的合作 | VTT 视角：VTT 如何描述与公司的合作 |
| **关键标识** | Company name + source_index | VAT id + index |

---

### 🔍 为什么需要两个数据源？

**互补性：**
- 📝 **数据源 1**：公司可能会在自己网站上强调某些合作
- 📝 **数据源 2**：VTT 可能会披露公司网站上没有提到的合作

**提高覆盖率：**
- ✅ 结合两个来源可以获得更完整的合作网络图
- ✅ 交叉验证：同一个合作关系在两个来源中都出现 → 更可信

---

### 📂 数据文件位置：

- **源 URL 列表**：`data/dataframes/comp_mentions_vtt_domain.csv`
- **原始图文档**：`data/graph_docs_vtt_domain/`
- **解析后图文档**：`data/graph_docs_vtt_domain_names_resolved/`

下面的代码将 VTT 域名来源的图文档转换为 DataFrame 表格。

In [21]:
# 📊 将 VTT 域名的图文档转换为 DataFrame
# 这是第二个数据源：来自 VTT 官方网站的合作信息
import pandas as pd
from tqdm import tqdm

# 创建空的 DataFrame
df_relationships_vtt_domain = pd.DataFrame(index=None)

# 📂 加载 VTT 域名的源数据
df_vtt_domain = pd.read_csv('data/dataframes/comp_mentions_vtt_domain.csv')

print(f"📌 数据源 2：VTT 域名")
print(f"   共有 {len(df_vtt_domain)} 个网页")
print(f"   来源：VTT 官方网站讨论合作的页面\n")

# 遍历所有 VTT 域名的网页
with tqdm(total=len(df_vtt_domain), desc="Processing VTT domain docs") as pbar:
    for index_source, row in df_vtt_domain.iterrows(): 
        try:     
            # 1️⃣ 加载图文档
            # 注意：文件名使用 VAT ID（而不是 Company name）
            Graph_Docs = pickle.load(
                open(os.path.join('data/graph_docs_vtt_domain_names_resolved/', 
                                  f"{row['Vat_id'].replace(' ','_')}_{index_source}.pkl"), 'rb')
            )[0]
                
            # 2️⃣ 构建节点映射字典
            node_description = {}
            node_en_id = {}
            for node in Graph_Docs.nodes:
                node_description[node.id] = node.properties['description']
                node_en_id[node.id] = node.properties['english_id']

            # 3️⃣ 提取所有关系
            relationship_rows = []
            for i in range(len(Graph_Docs.relationships)):
                rel = Graph_Docs.relationships[i]
                
                relationship_rows.append({
                    # 元信息（注意：这里使用 VAT id 而不是 Company name）
                    "Document number": index_source,
                    "VAT id": row["Vat_id"],  # 🔑 关键区别！
                    "relationship description": rel.properties['description'],
                    
                    # 起点节点
                    "source id": rel.source,
                    "source type": rel.source_type,
                    "source english_id": node_en_id.get(rel.source, None),
                    "source description": node_description.get(rel.source, None),
                    
                    # 关系类型
                    "relationship type": rel.type,
                    
                    # 终点节点
                    "target id": rel.target,
                    "target type": rel.target_type,
                    "target english_id": node_en_id.get(rel.target, None),
                    "target description": node_description.get(rel.target, None),
                    
                    # 原始来源（注意：字段名不同）
                    "Link Source Text": row["source_url"],      # VTT 网站的 URL
                    "Source Text": row["main_body"],            # 网页正文内容
                })

            # 4️⃣ 合并到总 DataFrame
            df_relationships_vtt_domain = pd.concat([
                df_relationships_vtt_domain, 
                pd.DataFrame(relationship_rows, index=None)
            ], ignore_index=True)

        except:
            # 文件不存在或读取失败，跳过
            continue
        
        pbar.update(1)

print(f"\n✅ 处理完成！")
print(f"   提取的关系数量：{len(df_relationships_vtt_domain)}")

# 5️⃣ 显示结果
df_relationships_vtt_domain.head(5)

📌 数据源 2：VTT 域名
   共有 1387 个网页
   来源：VTT 官方网站讨论合作的页面



Processing VTT domain docs:  84%|████████▎ | 1159/1387 [00:01<00:00, 717.42it/s]


✅ 处理完成！
   提取的关系数量：9185


,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text
0,0,FI10292588,"FiR 1 nuclear research reactor was developed, ...",FiR 1,Innovation,FiR 1,FiR 1 is a Triga-type nuclear research reactor...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...
1,0,FI10292588,Centre for Nuclear Safety is being developed a...,Centre for Nuclear Safety,Innovation,Centre for Nuclear Safety,A modern research facility under construction ...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...
2,3,FI08932048,The innovation approach 'Beyond the obvious' i...,Beyond the obvious,Innovation,Beyond the obvious,An innovation approach promising to provide so...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd,A visionary research and innovation partner fo...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...
3,4,FI01111693,Data-Driven Bioeconomy project is developed by...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland,A Finnish research and innovation partner work...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...
4,4,FI01111693,Data-Driven Bioeconomy project's forestry pilo...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,temp_1141,Organization,MHG Systems,An organization leading pilots developing fore...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...


#### 🔄 两个数据源的代码对比

**关键区别：**

```python
# 数据源 1️⃣：公司域名
df_relationships_comp_url
├─ CSV: 'vtt_mentions_comp_domain.csv'
├─ 图文档: 'graph_docs_names_resolved/'
├─ 文件命名: f"{Company_name}_{index}.pkl"
└─ 元数据列: "Source Company"

# 数据源 2️⃣：VTT 域名
df_relationships_vtt_domain
├─ CSV: 'comp_mentions_vtt_domain.csv'
├─ 图文档: 'graph_docs_vtt_domain_names_resolved/'
├─ 文件命名: f"{VAT_id}_{index}.pkl"
└─ 元数据列: "VAT id"
```

**数据示例对比：**

| 字段 | 数据源 1 (公司域名) | 数据源 2 (VTT 域名) |
|------|------------------|------------------|
| 标识符 | Company name: "Nokia" | VAT id: "FI12345678" |
| URL | www.nokia.com/... | www.vtt.fi/... |
| 文本字段 | text_content | main_body |
| 视角 | 公司如何描述与 VTT 的合作 | VTT 如何描述与公司的合作 |

In [22]:
# 🔍 实例：查看两个数据源的区别

print("=" * 70)
print("📊 数据源对比")
print("=" * 70)

# 数据源 1：公司域名
print("\n1️⃣ 数据源 1 - 公司域名 (vtt_mentions_comp_domain.csv)")
print("-" * 70)
df_comp = pd.read_csv('data/dataframes/vtt_mentions_comp_domain.csv')
print(f"   网页数量: {len(df_comp)}")
print(f"   列名: {list(df_comp.columns)[:5]}...")
print(f"\n   示例 URL: {df_comp['Link'].iloc[0]}")
print(f"   示例公司: {df_comp['Company name'].iloc[0]}")

# 数据源 2：VTT 域名
print("\n2️⃣ 数据源 2 - VTT 域名 (comp_mentions_vtt_domain.csv)")
print("-" * 70)
df_vtt = pd.read_csv('data/dataframes/comp_mentions_vtt_domain.csv')
print(f"   网页数量: {len(df_vtt)}")
print(f"   列名: {list(df_vtt.columns)[:5]}...")
print(f"\n   示例 URL: {df_vtt['source_url'].iloc[0]}")
print(f"   示例 VAT ID: {df_vtt['Vat_id'].iloc[0]}")

print("\n" + "=" * 70)
print("💡 关键洞察：")
print("=" * 70)
print("""
🔹 数据源 1：公司的视角
   - 公司在自己网站上如何描述与 VTT 的合作
   - URL 来自各个公司的官网
   - 可能会强调对自己有利的合作内容

🔹 数据源 2：VTT 的视角
   - VTT 在自己网站上如何披露与公司的合作
   - URL 都来自 vtt.fi 域名
   - 可能包含更多技术细节和研究成果

🔹 结合两者：
   - 获得更完整的合作网络图
   - 可以发现只在一方披露的合作关系
   - 交叉验证提高数据可信度
""")

📊 数据源对比

1️⃣ 数据源 1 - 公司域名 (vtt_mentions_comp_domain.csv)
----------------------------------------------------------------------
   网页数量: 1349
   列名: ['Orbis ID', 'Company name', 'Website', 'Link', 'Title']...

   示例 URL: https://www.fortum.com/media/2020/04/fortum-awarded-contract-decommissioning-vtts-nuclear-research-reactor-finland
   示例公司: FORTUM OYJ

2️⃣ 数据源 2 - VTT 域名 (comp_mentions_vtt_domain.csv)
----------------------------------------------------------------------
   网页数量: 1387
   列名: ['Vat_id', 'source_url', 'date_published', 'main_body', 'date_in_text']...

   示例 URL: https://www.vttresearch.com/en/news-and-ideas/finlands-old-nuclear-research-reactor-be-decommissioned-new-centre-nuclear-safety
   示例 VAT ID: FI10292588

💡 关键洞察：

🔹 数据源 1：公司的视角
   - 公司在自己网站上如何描述与 VTT 的合作
   - URL 来自各个公司的官网
   - 可能会强调对自己有利的合作内容

🔹 数据源 2：VTT 的视角
   - VTT 在自己网站上如何披露与公司的合作
   - URL 都来自 vtt.fi 域名
   - 可能包含更多技术细节和研究成果

🔹 结合两者：
   - 获得更完整的合作网络图
   - 可以发现只在一方披露的合作关系
   - 交叉验证提高数据可信度



#### 🎯 当前 Cell 功能总结

**核心功能：** 将 VTT 域名来源的图文档转换为 DataFrame（与前面处理公司域名的逻辑相同，但数据源不同）

---

### 📋 处理流程（与前面完全相同）：

```
1. 加载 CSV ──→ 'comp_mentions_vtt_domain.csv'
                 (包含 VTT 网站上提到合作公司的网页)
                 
2. 遍历每行 ──→ 读取对应的 PKL 图文档
                 文件名格式: {VAT_id}_{index}.pkl
                 
3. 提取关系 ──→ 每个关系转换为 DataFrame 的一行
                 包含 source, target, relationship type 等
                 
4. 合并数据 ──→ 所有关系合并到 df_relationships_vtt_domain
                 
5. 显示结果 ──→ df_relationships_vtt_domain.head(5)
```

---

### 🔑 与前面 Cell 的区别：

| 方面 | 前面的 Cell | 当前 Cell |
|------|-----------|----------|
| **变量名** | `df_relationships_comp_url` | `df_relationships_vtt_domain` |
| **源 CSV** | `vtt_mentions_comp_domain.csv` | `comp_mentions_vtt_domain.csv` |
| **图文档目录** | `graph_docs_names_resolved/` | `graph_docs_vtt_domain_names_resolved/` |
| **文件命名** | `{Company_name}_{i}.pkl` | `{Vat_id}_{index_source}.pkl` |
| **标识列** | "Source Company" | "VAT id" |
| **URL 列** | row["Link"] | row["source_url"] |
| **文本列** | row["text_content"] | row["main_body"] |

---

### 💡 为什么代码几乎相同？

因为**转换逻辑是一样的**：都是从图结构转为表格。
唯一的区别是**输入数据的来源和字段名**不同。

**最终目标：** 得到两个 DataFrame，后续可以合并分析：
- ✅ `df_relationships_comp_url` - 公司视角的合作关系
- ✅ `df_relationships_vtt_domain` - VTT 视角的合作关系

#### assess to OpenAI endpoint
- for this challenge we want to provide you access to OpenAI models: 4o-mini, 4.1 or 4.1-mini
- `ASK @ VTT-stand for key :)`

In [24]:
# 4. load api access credentials 
from langchain_openai import AzureChatOpenAI
import json

# initialize
model = initialize_llm(deployment_model= 'gpt-4o-mini')
# model = initialize_llm(deployment_model= 'gpt-4.1-mini', config_file_path= 'data/keys/azure_config.json')
# model = initialize_llm(deployment_model= 'gpt-4.1', config_file_path= 'data/keys/azure_config.json')

# example use:
prompt = ''
model.invoke(prompt).content

✅ 从 .env 加载配置:
   模型: gpt-4o-mini
   端点: https://lijie-mazglg3v-eastus2.cognitiveservices.azure.com/
   API 版本: 2025-01-01-preview


'Hello! How can I assist you today?'